In [6]:
import threading
import socket
import pickle

In [7]:
import time
import hashlib
import json
import uuid

NANOSECONDS = 1
MICROSECONDS = 1000 * NANOSECONDS
MILLISECONDS = 1000 * MICROSECONDS
SECONDS = 1000 * MILLISECONDS

MINE_RATE = 4 * SECONDS

STARTING_BALANCE = 1000

MINING_REWARD = 50
MINING_REWARD_INPUT = { 'address': 'MR' }

In [8]:
class TransactionPool:
    def __init__(self):
        self.transaction_map = {}

    def set_transaction(self, transaction):
        self.transaction_map[transaction.id] = transaction


    def existing_transaction(self, address):
        for transaction in self.transaction_map.values():
            if transaction.input['address'] == address:
                return transaction
    
    def transaction_data(self):
        return list(map(
            lambda transaction: transaction.to_json(),
            self.transaction_map.values()
        ))

    def clear_blockchain_transactions(self, blockchain):
        for block in blockchain.chain:
            for transaction in block.data:
                try:
                    del self.transaction_map[transaction['id']]
                except KeyError:
                    pass

In [19]:
class ManagerPeer:
    def __init__(self, ip):
        self.peer_set = set()
        self.MANAGER_IP = ip
        self.MANAGER_PORT = 8827
        self.server_port = 5527
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server_thread = threading.Thread(target=self.start_server)
        self.server_thread.daemon = True
        self.server_thread.start()
        self.stop_server = False
        self.transaction_pool = pickle.dumps(TransactionPool())
        self.validators = dict()
        
    def start_server(self):
        self.server_socket.bind((self.MANAGER_IP, self.MANAGER_PORT))
        self.server_socket.listen(100)
        print(f"Listening at port {self.MANAGER_PORT}")
        while True:
            if self.stop_server:
                break

            client_peer, address = self.server_socket.accept()
            print(f"incoming connection from {address}")
            recieved_request = client_peer.recv(4096).decode('utf-8')            

            if recieved_request.startswith("add"):
                ip = recieved_request.split()[1]
                self.inform_peers(recieved_request)
                client_peer.send(pickle.dumps(self.peer_set))
                self.peer_set.add(ip)
                print(f"added {ip}")

            elif recieved_request.startswith('remove'):
                ip = recieved_request.split()[1]
                self.peer_set.remove(ip)
                self.inform_peers(recieved_request)
                print(f"removed {ip}")

            elif recieved_request.startswith('update-transactionpool'):
                client_peer.send(b'Ok')
                self.transaction_pool = client_peer.recv(4096)
                
            elif recieved_request.startswith('get-transactionpool'):
                client_peer.send(self.transaction_pool)
            
            elif recieved_request.startswith('get-validators'):
                client_peer.send(pickle.dumps(self.validators))
                
            elif recieved_request.startswith('update-validator'):
                _, ip , amt = recieved_request.split(' ')
                self.validators[ip] = int(amt)
                
            client_peer.close()
        self.server_socket.close()
    
    def _stop_server(self):
        self.stop_server = True
    
    def inform_peers(self, message):
        operation, new_peer = message.split()
        for peer in self.peer_set:
            client_socket = socket.socket()
            client_socket.settimeout(30)
            try:
                client_socket.connect((peer, self.server_port))
                client_socket.send(f"{operation}-peer {new_peer}".encode('utf-8'))
                print(f"Sent to {peer}")
            except Exception as e:
                print(e)
                print(f"{operation}-peer {peer}")
            finally:
                client_socket.close()
                
    def __repr__(self):
        return f'{self.MANAGER_IP} {self.MANAGER_PORT}\nPeers: '+' '.join(self.peer_set)
    
    

In [20]:
manager_peer = ManagerPeer('192.168.43.228')

Listening at port 8827
incoming connection from ('192.168.43.228', 64390)
added 127.0.0.1
incoming connection from ('192.168.43.228', 64391)
incoming connection from ('192.168.43.228', 64392)
incoming connection from ('192.168.43.228', 64393)
incoming connection from ('192.168.43.228', 64921)
Sent to 127.0.0.1
added 192.168.43.228
incoming connection from ('192.168.43.228', 64923)
incoming connection from ('192.168.43.228', 64924)
incoming connection from ('192.168.43.228', 64926)


incoming connection from ('192.168.43.154', 57777)
added 192.168.43.154
incoming connection from ('192.168.43.154', 57778)


In [162]:
print(manager_peer.peer_set) 

set()
incoming connection from ('192.168.43.203', 63439)
added 192.168.43.203
incoming connection from ('192.168.43.203', 63440)
incoming connection from ('192.168.43.93', 52171)
Sent to 192.168.43.203
added 192.168.43.93
incoming connection from ('192.168.43.93', 52172)
incoming connection from ('192.168.43.203', 63443)


Exception in thread Thread-46:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-160-32f26cd7a325>", line 24, in start_server
    recieved_request = client_peer.recv(4096).decode('utf-8')
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 22: invalid start byte



In [42]:
# manager_peer._stop_server()

In [106]:
manager_peer

192.168.43.145 8825
Peers: 192.168.43.93 192.168.43.203

In [53]:
import jovian

<IPython.core.display.Javascript object>

In [ ]:
jovian.commit()

[jovian] Saving notebook..


<IPython.core.display.Javascript object>